In [1]:
import logging
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [2]:
# Create and configure the kernel.
from search import SearchPlugin

kernel = Kernel()

kernel.add_plugin(SearchPlugin(), plugin_name="search")

# The filter is used for demonstration purposes to show the function invocation.
#kernel.add_filter("function_invocation", function_invocation_filter)

KernelPlugin(name='search', description=None, functions={'search': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='search', plugin_name='search', description='Searches the internet for information.', parameters=[KernelParameterMetadata(name='query', description='The input text', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The input text'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='Returns the results from the internet search.', default_value=None, type_="<module 'json' from '/usr/local/lib/python3.12/json/__init__.py'>", is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'Returns the results from the internet search.'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyH

In [3]:
# Create the Coordinator agent. NOTE:The name of the agent must not contain spaces.
search_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o-mini"),
    name="SearchAgent",
    instructions=(
    """
    Your role is to search for information using Search to find relevant data and provide a summary of the results.
    """),
    plugins=[
        SearchPlugin(),
    ],
)

In [4]:
thread: ChatHistoryAgentThread = None

response = await search_agent.get_response(messages="Who won the 2025 NCAA Mens Basketball Championship?", thread=thread)

if response:
    print(f"Agent :> {response}")

await thread.delete() if thread else None

Performing search...
Query: 2025 NCAA Men's Basketball Championship winner
Agent :> The 2025 NCAA Men's Basketball Championship was won by the Florida Gators, who defeated the Houston Cougars with a score of 65-63. This victory marked Florida's first national championship since 2007. For further details, you can check out the articles from [USA Today](https://www.usatoday.com/story/sports/ncaab/2025/04/08/who-won-march-madness-2025-florida-houston-score/82986367007/) and [Bleacher Report](https://bleacherreport.com/articles/25182096-winners-and-losers-2025-mens-ncaa-tournament).
